# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_df = pd.read_csv('../output_data/Cities.csv')
#Adjusting cities_df
del cities_df['Unnamed: 0']
cities_df['Date'] = pd.to_datetime(cities_df['Date'], unit='s')
cities_df

,City_ID,Country,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,hu,csengele,46.542336,19.863584,61.00,52.0,18.0,4.36,2021-09-14 22:19:39
1,1,ir,delijan,33.990500,50.683800,78.12,12.0,0.0,4.12,2021-09-14 22:21:34
2,2,co,colon,1.191167,-76.968500,63.88,72.0,7.0,5.26,2021-09-14 22:21:36
3,3,no,lyngseidet,69.583333,20.216667,42.89,96.0,100.0,3.20,2021-09-14 22:21:37
4,4,pl,trzebiatow,54.059404,15.279377,50.27,87.0,1.0,6.31,2021-09-14 22:21:39
...,...,...,...,...,...,...,...,...,...,...
565,595,br,bom jardim,-7.016667,-46.483333,69.33,94.0,31.0,9.37,2021-09-14 22:33:20
566,596,sk,dunajska streda,47.983333,17.616667,69.66,78.0,1.0,5.97,2021-09-14 22:33:22
567,597,it,vignola,44.483333,11.000000,72.81,55.0,66.0,5.84,2021-09-14 22:33:24
568,598,ro,tulca,46.783333,21.783333,62.64,63.0,3.0,0.00,2021-09-14 22:33:25


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)
locations = cities_df[['Latitude','Longitude']]
weights = cities_df['Humidity']
highest_humidity = cities_df['Humidity'].max()
figure_layout = {'width':'800px', 'height':'400px','border':'1px solid black','padding':'1px','margin':'0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights,max_intensity=highest_humidity, dissipating = True)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
ideal_df = cities_df.loc[(cities_df['Max Temperature']<80) & (cities_df['Max Temperature']>70)
                        & (cities_df['Wind Speed']<10) & (cities_df['Cloudiness']==0)]
ideal_df

,City_ID,Country,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
1,1,ir,delijan,33.990500,50.683800,78.12,12.0,0.0,4.12,2021-09-14 22:21:34
7,7,bw,sefophe,-22.183333,27.966667,71.71,29.0,0.0,6.58,2021-09-14 22:21:42
26,28,it,cervia,44.261667,12.358889,73.18,67.0,0.0,5.75,2021-09-14 22:22:05
47,50,it,forli,44.223611,12.052778,73.42,61.0,0.0,3.44,2021-09-14 22:22:31
55,61,ir,tabas,33.595856,56.924368,79.52,8.0,0.0,3.83,2021-09-14 22:22:43
110,118,it,triggiano,41.066667,16.916667,72.32,72.0,0.0,5.75,2021-09-14 22:23:53
142,150,gr,kalamata,37.038889,22.114167,77.11,42.0,0.0,2.30,2021-09-14 22:24:29
303,325,fr,decines-charpieu,45.768991,4.972767,74.01,68.0,0.0,5.75,2021-09-14 22:27:55
311,333,am,mrgashat,40.129722,44.070556,75.20,45.0,0.0,2.95,2021-09-14 22:28:06
440,468,cn,haikou,20.045833,110.341667,78.93,100.0,0.0,2.24,2021-09-14 22:25:58


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Test google places api to search for hotels with a city in the dataframe (Sefophe,bw)
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
city_location = ('-22.183333,27.966667')
params = {'location':city_location,'radius':5000,'type':'hotel','language':'en', 'key':g_key}
response = requests.get(url, params).json()
hotel_name = response['results'][0]['name']
# hotel_latitude = response['results'][0]['geometry']['location']['lat']
# hotel_longitude = response['results'][0]['geometry']['location']['lng']
hotel_name

'Sefhophe'

In [12]:
hotel_df = ideal_df
hotel_df.loc[:,'Hotel Name'] = ''
params = {'radius':5000,'type':'hotel','language':'en', 'key':g_key}
for index,row in hotel_df.iterrows():
    latitude = str(row['Latitude'])
    longitude = str(row['Longitude'])
    hotel_location = (f'{latitude},{longitude}')
    params['location'] = hotel_location
    response = requests.get(url, params).json()
    hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
hotel_df


C:\Users\tamer\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City_ID,Country,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
1,1,ir,delijan,33.990500,50.683800,78.12,12.0,0.0,4.12,2021-09-14 22:21:34,Delijan
7,7,bw,sefophe,-22.183333,27.966667,71.71,29.0,0.0,6.58,2021-09-14 22:21:42,Sefhophe
26,28,it,cervia,44.261667,12.358889,73.18,67.0,0.0,5.75,2021-09-14 22:22:05,Cervia
47,50,it,forli,44.223611,12.052778,73.42,61.0,0.0,3.44,2021-09-14 22:22:31,Forli
55,61,ir,tabas,33.595856,56.924368,79.52,8.0,0.0,3.83,2021-09-14 22:22:43,Tabas
110,118,it,triggiano,41.066667,16.916667,72.32,72.0,0.0,5.75,2021-09-14 22:23:53,Capurso
142,150,gr,kalamata,37.038889,22.114167,77.11,42.0,0.0,2.30,2021-09-14 22:24:29,Kalamata
303,325,fr,decines-charpieu,45.768991,4.972767,74.01,68.0,0.0,5.75,2021-09-14 22:27:55,Villeurbanne
311,333,am,mrgashat,40.129722,44.070556,75.20,45.0,0.0,2.95,2021-09-14 22:28:06,Armavir
440,468,cn,haikou,20.045833,110.341667,78.93,100.0,0.0,2.24,2021-09-14 22:25:58,Haikou


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer((markers))
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…